# **<span style="color:#344ceb;"> Library Installations</span>**

In [1]:
!pip install keras_preprocessing
!pip install tensorflow-addons
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=176b96e734aa79b1046f6da0d693c0dcb9d86d636ebea5b993ecaf741fbaa69d
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


# **<span style="color:#344ceb;">Library Import</span>**

In [2]:
import pandas as pd
import numpy as np
from time import sleep
from tqdm import tqdm
import keras
import tensorflow as tf
from keras.layers import Input
from keras.layers import Embedding
from keras.layers import Bidirectional
from keras.layers import LSTM
from keras import regularizers
from keras.layers import TimeDistributed
from keras.layers import Dropout
from keras.layers import Dense
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from keras.initializers import RandomUniform
from keras import regularizers
from keras.optimizers import Adam
from keras.models import Sequential, Model
from keras.layers import Conv1D
from keras.layers import Flatten
from keras.layers import concatenate
from keras.layers import MaxPooling1D
from keras.layers import InputLayer
from tensorflow_addons.layers import CRF
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow_addons.text.crf_wrapper import CRFModelWrapper


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
train = pd.read_csv("/kaggle/input/voc-processed-data/train-nl.tsv", sep="\t")
test = pd.read_csv("/kaggle/input/voc-processed-data/test-nl.tsv", sep="\t")
dev = pd.read_csv("/kaggle/input/voc-processed-data/dev-nl.tsv", sep="\t")

data = pd.concat([train, test, dev], ignore_index=True)
print(len(data))

506867


In [4]:
def preprocess_data(data):
    data['Sentence'] = data['TOKEN'].str.extract(r'# document_path = (.*)', expand=False).ffill()
    data = data.dropna()

    # create a dictionary to map unique strings to Sentence numbers
    sentence_dict = {}
    count = 1
    for sentence in data['Sentence'].unique():
        sentence_dict[sentence] = count
        count += 1

    # map the 'Sentence #' column with the dictionary
    data['Sentence'] = data['Sentence'].map(sentence_dict)
    data = data.reset_index(drop=True)
    
    return data

In [5]:
df = preprocess_data(data)
df = df[['Sentence', 'TOKEN', 'NE-MAIN']]

df.rename(columns={"TOKEN": "Word","NE-MAIN":"Tag"},inplace=True)
df.head()

/tmp/ipykernel_310/4232299772.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Sentence'] = data['Sentence'].map(sentence_dict)


,Sentence,Word,Tag
0,1,heste,O
1,1,afschrift,O
2,1,m,O
3,1,In,O
4,1,den,O


In [6]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from keras_preprocessing.sequence import pad_sequences


class ContextNER:

    __X, __y = None, None
    X_array, y_array = None, None

    word2idx, idx2word = None, None
    tag2idx, idx2tag = None, None
    y_array_normal = None

    def __init__(self, df, all_words, max_len=None):

        self.__df = df

        self.all_words = set(all_words)
        self.all_tags = set(df.Tag.values)

        self.sentences = self.__build_sentences()

        self.num_words = len(self.all_words) + 2
        self.num_tags = len(self.all_tags) + 1

        if max_len:
            self.max_len = max_len
        else:
            self.max_len = self._get_maxlen()

        self.__build_Xy()
        self.__build_parsers()
        self.__parser_arrays()

    def _get_maxlen(self):
        return max([len(x) for x in self.sentences]) + 1

    def __build_sentences(self):

        return [x for x in self.__df.groupby('Sentence').apply(
            lambda xdef: [x for x in zip(
                xdef.Word.values,
                xdef.Tag.values
            )]
        )]

    def __build_Xy(self):

        self.__X = [[word for word, __ in value] for value in self.sentences]
        self.__y = [[tag for __, tag in value] for value in self.sentences]

    def __build_parsers(self):

        self.word2idx = {value: idx + 2 for idx,
                         value in enumerate(self.all_words)}
        self.word2idx["UNK"] = 1  
        self.word2idx["PAD"] = 0  

        # Converte um index em Word
        self.idx2word = {idx: value for value, idx in self.word2idx.items()}

        # Converte Tag em ìndice
        self.tag2idx = {value: idx + 1 for idx,
                        value in enumerate(self.all_tags)}
        self.tag2idx["PAD"] = 0  

        self.idx2tag = {idx: value for value, idx in self.tag2idx.items()}

    def parser2categorical(self, y_pred, y_true):

        pred_tag = [[self.idx2tag[idx] for idx in row] for row in y_pred]
        y_true_tag = [[self.idx2tag[idx] for idx in row] for row in y_true]

        return pred_tag, y_true_tag

    def __parser_arrays(self):

        tmp_X = [[self.word2idx[index] for index in value]
                 for value in self.__X]
        tmp_y = [[self.tag2idx[index] for index in value]
                 for value in self.__y]

        self.X_array = pad_sequences(maxlen=self.max_len,
                                     sequences=tmp_X,
                                     padding="post",
                                     value=0)

        y_pad = pad_sequences(maxlen=self.max_len,
                              sequences=tmp_y,
                              padding="post",
                              value=0)

        self.y_array_normal = y_pad
        self.y_array = np.array(
            [to_categorical(index, num_classes=self.num_tags, dtype='int8') for index in y_pad])


In [7]:
def build_matrix_embeddings(path, num_tokens, embedding_dim, word_index):
    """
        Function to load pre-trained files into memory
        
    """

    hits, misses = 0, 0
    embeddings_index = {}

    print('Loading file...')

    sleep(0.5)

    for line in tqdm(open(path, encoding='utf-8')):
        word, coefs = line.split(maxsplit=1)
        embeddings_index[word] = np.fromstring(coefs, "f", sep=" ")

    print("Found %s Word Vectors." % len(embeddings_index))

    sleep(0.5)

    # Prepare embedding matrix
    embedding_matrix = np.zeros((num_tokens, embedding_dim))

    for word, i in tqdm(word_index.items()):
        if i >= num_tokens:
            continue
        try:
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
                hits += 1
            else:
                embedding_vector = embeddings_index.get(str(word).lower())
                if embedding_vector is not None:
                    embedding_matrix[i] = embedding_vector
                    hits += 1
                else:
                    embedding_vector = embeddings_index.get(str(word).upper())
                    if embedding_vector is not None:
                        embedding_matrix[i] = embedding_vector
                        hits += 1
                misses += 1
        except:
            embedding_matrix[i] = embeddings_index.get('UNK')

    print("Converted: %d Tokens | Lost: %d Tokens" % (hits, misses))

    return embedding_matrix


In [8]:
ner_aux = ContextNER(df, df.Word.to_list())

<a id="6"></a>
## <p style="background-color:#344ceb;font-family:Serif;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;"> Loading the pretrained Embedding</p>
#### 

In [9]:
%%time
file_path = '/kaggle/input/glove6b100dtxt/glove.6B.100d.txt'

glove_embeddings = \
build_matrix_embeddings(path=file_path,
                        num_tokens=ner_aux.num_words, 
                        embedding_dim=100,
                        word_index=ner_aux.word2idx)

Loading file...


400000it [00:14, 28281.31it/s]


Found 400000 Word Vectors.


100%|██████████| 68355/68355 [00:00<00:00, 410049.44it/s]


Converted: 9102 Tokens | Lost: 62991 Tokens
CPU times: user 13.3 s, sys: 517 ms, total: 13.8 s
Wall time: 15.4 s


In [10]:
from sklearn.model_selection import train_test_split

X_TRAIN, X_TEST, Y_TRAIN, Y_TEST  = train_test_split(ner_aux.X_array,ner_aux.y_array,test_size=0.3, random_state=42)
X_TEST, X_VAL, Y_TEST, Y_VAL = train_test_split(X_TEST, Y_TEST, test_size=1/3, random_state=42)

In [11]:
print(np.shape(X_TRAIN))
print(np.shape(X_TEST))
print(np.shape(X_VAL))

(1539, 704)
(440, 704)
(220, 704)


In [12]:
X_TRAIN = np.asarray(X_TRAIN).astype(np.float32)
X_TEST = np.asarray(X_TEST).astype(np.float32)
X_VAL = np.asarray(X_VAL).astype(np.float32)

Y_TRAIN = np.asarray(Y_TRAIN).astype(np.float32)
Y_TEST = np.asarray(Y_TEST).astype(np.float32)
Y_VAL = np.asarray(Y_VAL).astype(np.float32)

<a id="6"></a>
## <p style="background-color:#344ceb;font-family:Serif;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;"> Char Embeddings</p>
#### 

In [13]:
#Chars Embeddings/features
def get_X_char(sentences, max_len,  max_len_char, char2idx):
    X_char = []
    for sentence in tqdm(sentences):
        sent_seq = []
        for i in range(max_len):
            word_seq = []
            for j in range(max_len_char):
                try:
                    word_seq.append(char2idx.get(sentence[i][0][j]))
                except:
                    word_seq.append(char2idx.get("PAD"))
            sent_seq.append(word_seq)
        X_char.append(np.array(sent_seq))

    return np.asarray(X_char)

In [14]:
def save_best_model(path_folder, file_name, metric):

    if metric == 'f1':
        mode = 'max'
        
    elif metric == 'val_loss':
        mode = 'min'
        
    return ModelCheckpoint(path_folder + file_name + '.h5',
                           monitor=metric,
                           mode=mode,
                           verbose=1,
                           save_best_only=True)

In [15]:
def show_true_pred(index, true_tag, pred_tag, idx2word, ner_source):

    word_parser_idx = idx2word
    TEST = ner_source[index]

    print("{:25}   {:10}   {}".format("Word", "True", "Pred"))
    print("=" * 50)

    for word, true, pred in zip([word_parser_idx[x] for x in TEST],
                                 true_tag[index],
                                 pred_tag[index]):
        if pred != 'PAD':
            print("{:25}   {:10}    {}".format(word, true, pred))

In [16]:
chars = set([w_i for w in ner_aux.all_words for w_i in str(w)])
n_chars = len(chars) + 2
max_len_char = 10

In [17]:
char2idx = {c: i + 2 for i, c in enumerate(chars)}
char2idx["UNK"] = 1
char2idx["PAD"] = 0
idx2char = {i: c for c, i in char2idx.items()}

In [18]:
X_train_char_X = [[ner_aux.idx2word[x] for x in sent] for sent in X_TRAIN]
X_test_char_X = [[ner_aux.idx2word[x] for x in sent] for sent in X_TEST]
X_val_char_X = [[ner_aux.idx2word[x] for x in sent] for sent in X_VAL]

In [20]:
X_TRAIN_CHAR = get_X_char(X_train_char_X, ner_aux.max_len, max_len_char, char2idx)
X_TEST_CHAR = get_X_char(X_test_char_X, ner_aux.max_len, max_len_char, char2idx)
X_VAL_CHAR = get_X_char(X_val_char_X, ner_aux.max_len, max_len_char, char2idx)

100%|██████████| 220/220 [00:01<00:00, 152.07it/s]


# **<span style="color:#344ceb;"> NER MODELING</span>**

In [21]:
def BiLSTM_CNN(isa_crf=True,words_weights=None,pre_trained=False,max_len=None,max_len_char=None,
               num_words=None,num_tags=None,n_chars=None,kernel_size=None,filter_size=None,
               hiden_layer=128,is_trainable=False,l2_reg=1e-6,learning_rate=0.001,dropout=0.2):

    # Char Embbedings
    chars_input = Input(shape=(max_len, max_len_char,), dtype='float32')
    chars_embed = Embedding(input_dim=n_chars,
                            output_dim=max_len_char,
                            embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5))(chars_input)

    chars_embed = TimeDistributed(Dropout(rate=dropout))(chars_embed)

    # Cnn Features
    chars_embed = TimeDistributed(Conv1D(kernel_size=kernel_size,
                                         filters=filter_size,
                                         padding='same',
                                         activation='tanh',
                                         strides=1))(chars_embed)

    chars_embed = TimeDistributed(MaxPooling1D(
        pool_size=max_len_char))(chars_embed)
    chars_embed = TimeDistributed(Flatten())(chars_embed)

    # Word Embbedings
    words_input = Input(shape=(max_len,), dtype='float32')
    words_embed = Embedding(input_dim=words_weights.shape[0],
                            output_dim=words_weights.shape[1],
                            weights=[words_weights],
                            trainable=is_trainable)(words_input)

    x = concatenate([words_embed, chars_embed])

    BiLSTM = Bidirectional(LSTM(units=hiden_layer // 2,
                                return_sequences=True,
                                recurrent_dropout=0.1,
                                kernel_regularizer=regularizers.l2(l2_reg)))(x)

    BiLSTM = TimeDistributed(Dropout(rate=dropout))(BiLSTM)
    BiLSTM = TimeDistributed(Dense(units=hiden_layer,
                                   activation='relu'))(BiLSTM)

#     crf = CRF(num_tags)
#     decoded_sequence, potentials, sequence_length, chain_kernel = crf(BiLSTM)

    BiLSTM = Dense(units=num_tags, activation='softmax')(BiLSTM)
    
    model = Model(inputs=[words_input, chars_input],
                  outputs=BiLSTM)
    
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

#     model.compile(optimizer="rmsprop", loss='categorical_crossentropy',metrics=['accuracy'])
    
    return model


<a id="7"></a>
## <p style="background-color:#344ceb;font-family:Serif;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;"> Setting Hyper Parameters</p>
#### 

In [26]:
EPOCHS = 100
DROPOUT = 0.2
BATCH_SIZE = 64
LEARNING_RATE = 0.01
HIDEN_LAYER = 128
L2_REG = 1e-9
VALIDATION_SPLIT = 0.1

FILTER_SIZE = 30
KERNEL_SIZE = 3

In [27]:
model = BiLSTM_CNN(isa_crf=True,
                   words_weights=glove_embeddings,
                   pre_trained=True,
                   max_len=ner_aux.max_len,
                   max_len_char=max_len_char,
                   n_chars=n_chars,
                   num_words=ner_aux.num_words,
                   num_tags=ner_aux.num_tags,
                   kernel_size=KERNEL_SIZE,
                   filter_size=FILTER_SIZE,
                   hiden_layer=HIDEN_LAYER,
                   is_trainable=True,
                   l2_reg=L2_REG,
                   learning_rate=LEARNING_RATE,
                   dropout=DROPOUT)

<a id="7"></a>
## <p style="background-color:#344ceb;font-family:Serif;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;"> Model Summary</p>

In [ ]:
model.summary()

<a id="7"></a>
## <p style="background-color:#344ceb;font-family:Serif;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;"> Training the model</p>


In [ ]:
# from keras.utils import plot_model
# plot_model(model,to_file='model_bilstm_crf.png',
#            show_shapes=True,
#            show_layer_names=True,
#            expand_nested=True)

## 

In [28]:
%%time
callbacks = keras.callbacks.EarlyStopping(monitor='categorical_crossentropy', patience=5,
                         verbose=1, mode='max', restore_best_weights=True)

X_TRAIN_CHAR = np.asarray(X_TRAIN_CHAR).astype(np.float32)
X_VAL_CHAR = np.asarray(X_VAL_CHAR).astype(np.float32)
X_TEST_CHAR = np.asarray(X_TEST_CHAR).astype(np.float32)

History = \
model.fit([X_TRAIN, X_TRAIN_CHAR],
           Y_TRAIN, 
           batch_size=BATCH_SIZE, 
           epochs=EPOCHS, 
           validation_data=([X_VAL, X_VAL_CHAR],Y_VAL),
           callbacks=[callbacks])

Epoch 1/100
25/25 [==============================] - 141s 5s/step - loss: 0.3073 - accuracy: 0.9268 - val_loss: 0.1221 - val_accuracy: 0.9696
Epoch 2/100
25/25 [==============================] - 136s 5s/step - loss: 0.0960 - accuracy: 0.9737 - val_loss: 0.0840 - val_accuracy: 0.9763
Epoch 3/100
25/25 [==============================] - 136s 5s/step - loss: 0.0617 - accuracy: 0.9820 - val_loss: 0.0679 - val_accuracy: 0.9814
Epoch 4/100
25/25 [==============================] - 136s 5s/step - loss: 0.0426 - accuracy: 0.9874 - val_loss: 0.0612 - val_accuracy: 0.9824
Epoch 5/100
25/25 [==============================] - 137s 5s/step - loss: 0.0312 - accuracy: 0.9902 - val_loss: 0.0615 - val_accuracy: 0.9827
Epoch 6/100
25/25 [==============================] - 134s 5s/step - loss: 0.0233 - accuracy: 0.9925 - val_loss: 0.0656 - val_accuracy: 0.9820
Epoch 7/100
25/25 [==============================] - 136s 5s/step - loss: 0.0192 - accuracy: 0.9936 - val_loss: 0.0669 - val_accuracy: 0.9827
Epoch 

In [29]:
import numpy as np
from keras.callbacks import Callback
from seqeval.metrics import f1_score
from seqeval.metrics import recall_score
from seqeval.metrics import accuracy_score
from seqeval.metrics import precision_score
from seqeval.metrics import classification_report

class F1Metrics(Callback):

    def __init__(self, id2label, pad_value=0, validation_data=None, digits=4):

        super(F1Metrics, self).__init__()
        
        self.id2label = id2label
        self.pad_value = pad_value
        self.validation_data = validation_data
        self.digits = digits
        self.is_fit = validation_data is None

    def convert_idx_to_name(self, y, array_indexes):

        y = [[self.id2label[idx] for idx in row[row_indexes]] for
             row, row_indexes in zip(y, array_indexes)]
        return y

    def predict(self, X, y):

        y_pred = self.model.predict_on_batch(X)

        # reduce dimension.
        y_true = np.argmax(y, -1)
        y_pred = np.argmax(y_pred, -1)

        non_pad_indexes = [np.nonzero(y_true_row != self.pad_value)[0] for y_true_row in y_true]

        y_true = self.convert_idx_to_name(y_true, non_pad_indexes)
        y_pred = self.convert_idx_to_name(y_pred, non_pad_indexes)

        return y_true, y_pred

    def score(self, y_true, y_pred):
        
        f_score = f1_score(y_true, y_pred)
        r_score = recall_score(y_true, y_pred)
        p_score = precision_score(y_true, y_pred)
        
        print('NER Metrics > precision_score: {:04.2f}  --  recall_score: {:04.2f}  --  f1_score: {:04.2f}'.format(p_score, r_score, f_score))
        
        return f_score, r_score, p_score

    def on_epoch_end(self, epoch, logs={}):
        
        if self.is_fit:
            self.on_epoch_end_fit(epoch, logs)
        else:
            self.on_epoch_end_fit_generator(epoch, logs)

    def on_epoch_end_fit(self, epoch, logs={}):
        
        X = self.validation_data[0]
        y = self.validation_data[1]
        y_true, y_pred = self.predict(X, y)

        f_score,\
        r_score,\
        p_score = self.score(y_true, y_pred)
        
        logs['f1'], logs['recall'], logs['precision'] = f_score, r_score, p_score
        

    def on_epoch_end_fit_generator(self, epoch, logs={}):
        
        y_true = []
        y_pred = []
        
        for X, y in self.validation_data:
            y_true_batch, y_pred_batch = self.predict(X, y)
            y_true.extend(y_true_batch)
            y_pred.extend(y_pred_batch)

        f_score,\
        r_score,\
        p_score = self.score(y_true, y_pred)
        
        logs['f1'], logs['recall'], logs['precision'] = f_score, r_score, p_score
        
        
        
def all_metrics(pred_tag, true_tag):

    print(classification_report(pred_tag, true_tag))
    print('=' * 25)
    print("Precision: \t", precision_score(pred_tag, true_tag))
    print("Recall: \t", recall_score(pred_tag, true_tag))
    print("F1: \t\t", f1_score(pred_tag, true_tag))
    
def all_metrics_fold(pred_tag, true_tag):

    print('=' * 25)
    print("Precision: \t", precision_score(pred_tag, true_tag))
    print("Recall: \t", recall_score(pred_tag, true_tag))
    print("F1: \t\t", f1_score(pred_tag, true_tag))

<a id="9"></a>
## <p style="background-color:#344ceb;font-family:Serif;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;"> Evaluating the model</p>


In [30]:
X_TEST = np.asarray(X_TEST).astype(np.float32)
X_TEST_CHAR = np.asarray(X_TEST_CHAR).astype(np.float32)

In [31]:
del X_TRAIN, 
del X_TRAIN_CHAR
del Y_TRAIN

In [32]:
y_pred, y_true = \
np.argmax(model.predict([X_TEST, X_TEST_CHAR], verbose=1, batch_size=64), axis=-1), \
np.argmax(Y_TEST, -1)

pred_tag, true_tag = \
ner_aux.parser2categorical(y_pred, y_true) 

all_metrics(pred_tag, true_tag)

7/7 [==============================] - 4s 480ms/step


/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PAD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                               precision    recall  f1-score   support

                           AD       1.00      1.00      1.00       440
                 Organization       0.24      0.16      0.19       372
         Organization,B-Place       0.29      0.45      0.35        47
        Organization,I-Person       0.00      0.00      0.00         0
Organization,I-Person,B-Place       0.00      0.00      0.00         0
Organization,I-Person,I-Place       0.00      0.00      0.00         0
         Organization,I-Place       0.13      0.33      0.19        15
                       Person       0.49      0.48      0.48      2401
               Person,B-Place       0.46      0.60      0.52        30
               Person,I-Place       0.15      0.33      0.21         6
                        Place       0.40      0.40      0.40       865

                    micro avg       0.50      0.49      0.50      4176
                    macro avg       0.29      0.34      0.30      4176
    

In [33]:
def show_true_pred(index, true_tag, pred_tag, idx2word, ner_source):

    word_parser_idx = idx2word
    TEST = ner_source[index]

    print("{:25}   {:10}   {}".format("Word", "True", "Pred"))
    print("=" * 50)

    for word, true, pred in zip([word_parser_idx[x] for x in TEST[:50]],
                                 true_tag[index],
                                 pred_tag[index]):
        if pred != 'PAD':
            print("{:25}   {:10}    {}".format(word, true, pred))
            

<a id="10"></a>
## <p style="background-color:#344ceb;font-family:Serif;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;"> Result Report</p>


In [34]:
for index in range(2):
    show_true_pred(index, pred_tag, true_tag, ner_aux.idx2word, X_TEST)

Word                        True         Pred
.                           O             O
endutie                     O             O
Gehouden                    O             O
ken                         O             O
Holdij                      B-Place       B-Place
L:                          O             O
B:                          B-Person      O
Comptoir                    B-Place       B-Place
onder                       O             O
dato                        O             O
13.'                        O             O
februarij                   O             O
1759.                       O             O
n                           O             O
overleden                   O             O
1e                          O             O
Kretschmar                  B-Person      B-Person
han                         O             O
e                           O             O
gemeijnd                    O             O
volgt                       O             O
Dit        